Import Necessary libraries

In [ ]:
from pathlib import Path
import pydicom
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install torch torchvision torchaudio torchmetrics


In [ ]:
pip install numpy==1.22.0

In [ ]:
from torchvision import transforms

In [ ]:
import torch
import torchvision
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

Read the labels dataframe.

In [ ]:
labels = pd.read_csv('/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv')

In [ ]:
labels.head()

While looking at labels, we can see that there is a **patientid** column at first which is a unique patient id.

Columns`x,y, left, right` represents the `x` and `y` coordinates and the `left` and `right` position of the classified pneumonia region.

The final column `Target` is used to denote whether patient is suffering from pneumonia or not.

In [ ]:
# Dropping Duplicates

labels = labels.drop_duplicates("patientId")

Let's create a `ROOT_PATH` that takes a path to our train dataset.

Additionally, let's define a `SAVE_PATH` that defines path to the place where our processed images are saved.

In [ ]:
ROOT_PATH = Path("/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/")
SAVE_PATH = Path("Processed")

Let's look at some example images.

Let's create 3x3 subplots to view 9 images along with their labels.

In [ ]:
fig, axis = plt.subplots(3, 3, figsize = (9, 9))
c = 0
for i in range(3):
    for j in range(3):
        # Start reading the files.
        patient_id = labels.patientId.iloc[c]
        
        # Create a path to the dicom file of this particular patient/
        dcm_path = ROOT_PATH/patient_id
        
        # Add .dcm extension
        dcm_path = dcm_path.with_suffix(".dcm")
        
        # Reading the dicom file
        dcm = pydicom.read_file(dcm_path).pixel_array
        
        # Extract the labels of the particular patient from labels dataframe
        label = labels['Target'].iloc[c]
        
        # Visualize the image
        axis[i][j].imshow(dcm, cmap = "bone")
        axis[i][j].set_title(label)
        
        c += 1
        
         
        

###  Preprocessing Images

First, we standardize each image pixels by dividing it by 255.

Also, our images are way too large for current neural network architectures to process. So, we need to resize them to shape of 224x224.

To use less space when storing the images, we convert them to `float16`

In [ ]:
# Initialization 
sums, sums_squared = 0, 0

# Loop over all patient ids
# To decide if a data is used as training data or validation data
# We can use the enumerate 

for c, patient_id, in enumerate(tqdm(labels.patientId)):
    # Start reading the files.
    patient_id = labels.patientId.iloc[c]
        
    # Create a path to the dicom file of this particular patient/
    dcm_path = ROOT_PATH/patient_id

    # Add .dcm extension
    dcm_path = dcm_path.with_suffix(".dcm")

    # Reading the dicom file and dividing the pixel array by 255
    dcm = pydicom.read_file(dcm_path).pixel_array
    
    # Resizing the image and converting it's type
    dcm_array = cv2.resize(dcm, (224, 224)).astype(np.float16)
    
    # Storing the label in 'label'
    label = labels.Target.iloc[c]
    
    # Identify training/validation
    train_or_val = "train" if c < 24000 else "val"
    
    # Save preprocessed images
    current_save_path = SAVE_PATH/train_or_val/str(label)
    current_save_path.mkdir(parents = True, exist_ok = True)
    np.save(current_save_path/patient_id, dcm_array)
    
    
    # Update sums and sums_squared
    normalizer = 224*224
    

In [ ]:
mean = 0.49
std = 0.24

### Train and Validation Dataset

To load the generate the data in required format, we can make use of dataset class.

In [ ]:
def load_file(path):
    return np.load(path).astype(np.float32)

Next, we can define our train and validation transform:

In [ ]:
# Transforms

train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.RandomAffine(degrees = (-5, 5), translate = (0, 0.05), scale = (0.9, 1.1)),
    transforms.RandomResizedCrop((224, 224), scale = (0.35, 1))
])

val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
# DataLoader

train_dataset = torchvision.datasets.DatasetFolder("/kaggle/working/Processed/train/", loader = load_file, extensions = "npy", transform = train_transforms)

val_dataset = torchvision.datasets.DatasetFolder("/kaggle/working/Processed/val/", loader = load_file, extensions = "npy", transform = val_transforms)

In [ ]:
fig, axis = plt.subplots(2, 2, figsize=(9, 9))

for i in range(2):
    for j in range(2):
        random_index = np.random.randint(0, 24000)
        x_ray, label = train_dataset[random_index]
        axis[i][j].imshow(x_ray[0], cmap = "bone")
        axis[i][j].set_title(label)


In [ ]:
batch_size = 64
num_workers = 4

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, num_workers = num_workers, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, num_workers = num_workers, shuffle = False)

In [ ]:
np.unique(train_dataset.targets, return_counts = True)

Our dataset is heavily imbalanced. So, we will be doing weighted loss.

In [ ]:
# Observing the Resnet18 architecture

torchvision.models.resnet18()

In [ ]:
class PneumoniaModel(pl.LightningModule):
    
    # Constructor
    def __init__(self):
        super(PneumoniaModel, self).__init__()
        self.model = torchvision.models.resnet18()
        self.model.conv1 = torch.nn.Conv2d(in_channels = 1, out_channels=64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.model.fc = torch.nn.Linear(in_features = 512, out_features = 1, bias = True)
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = 1e-4)
        self.loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight = torch.tensor([3]))
        
        self.train_acc = torchmetrics.Accuracy(task = "binary")
        self.val_acc = torchmetrics.Accuracy(task = "binary")
        #self.test_acc = torchmetrics.Accuracy()
        
        
        
    # Activation
    def forward(self, data):
        pred = self.model(data)
        return pred
    
    def training_step(self, batch, batch_idx):
        x_ray, label = batch
        label = label.float()
        pred = self(x_ray)[:, 0]
        loss = self.loss_fn(pred, label)
        
        self.log("Train Loss", loss)
        self.log("Step Train ACC", self.train_acc(torch.sigmoid(pred), label.int()))
        
        return loss
    
    
    
    def on_train_epoch_end(self):
        self.log("Train ACC", self.train_acc.compute())
        
    
    def validation_step(self, batch, batch_idx):
        x_ray, label = batch
        label = label.float()
        pred = self(x_ray)[:, 0]
        loss = self.loss_fn(pred, label)
        
        self.log("Val Loss", loss)
        self.log("Step Val ACC", self.val_acc(torch.sigmoid(pred), label.int()))

        
        
       
    def on_validation_epoch_end(self):
        self.log("Val ACC", self.val_acc.compute())
        
        
    def configure_optimizers(self):
        return[self.optimizer]
    

In [ ]:
model = PneumoniaModel()

In [ ]:
checkpoint_callback = ModelCheckpoint(
    monitor = "Val ACC",
    save_top_k = 1,
    mode = "max",
    dirpath = "./saved_models"
)

In [ ]:
# Configure the trainer
trainer = pl.Trainer(logger = TensorBoardLogger(save_dir = "./logs"), log_every_n_steps = 64, callbacks = checkpoint_callback, max_epochs = 10)

In [ ]:
# Train the model
trainer.fit(model, train_loader, val_loader)

In [ ]:
# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = PneumoniaModel.load_from_checkpoint("/kaggle/working/saved_models/epoch=0-step=375.ckpt")

In [ ]:
model.eval()

In [ ]:
model.to(device)

In [ ]:
preds = []
labels = []

with torch.no_grad():
    for data, label in tqdm(val_dataset):
        data = data.to(device).float().unsqueeze(0)
        
        # Calculate probabilities
        pred = torch.sigmoid(model(data)[0].cpu())
        
        preds.append(pred)
        labels.append(label)
preds = torch.tensor(preds)
labels = torch.tensor(labels).int()

In [ ]:
# Accuracy
acc = torchmetrics.Accuracy(task = "binary")(preds, labels)

# Precision
precision = torchmetrics.Precision(task = "binary")(preds, labels)

# Recall
recall = torchmetrics.Recall(task = "binary")(preds, labels)

# Confusion Matrix
cm = torchmetrics.ConfusionMatrix(num_classes=2, task = "binary")(preds, labels)

In [ ]:
print(f"Accuracy : {acc}")
print(f"Precision : {precision}")
print(f"Recall : {recall}")

### Class Activation Map
Learning deep features for discriminative localization.

In [ ]:
def load_file(path):
    return np.load(path).astype(np.float32)

In [ ]:
val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.49, 0.248),
    transforms.Resize((224, 224))
])

In [ ]:
from torchvision.datasets import DatasetFolder

In [ ]:
val_dataset = torchvision.datasets.DatasetFolder("/kaggle/working/Processed/val/", loader = load_file, extensions = 'npy', transform = val_transforms)

In [ ]:
temp_model = torchvision.models.resnet18()

In [ ]:
list(temp_model.children())

In [ ]:
list(temp_model.children())[:-2]

In [ ]:
torch.nn.Sequential(*list(temp_model.children())[:-2])

In [ ]:
class PneumoniaModel(pl.LightningModule):
    
    def __init__(self):
        super(PneumoniaModel, self).__init__()
        self.model = torchvision.models.resnet18()
        self.model.conv1 = torch.nn.Conv2d(in_channels = 1, out_channels=64, kernel_size=(7, 7), stride = (2, 2), padding = (3, 3), bias = False)
        self.model.fc = torch.nn.Linear(in_features=512, out_features=1)
        
        self.feature_map = torch.nn.Sequential(*list(temp_model.children())[:-2])
        
        
    def forward(self, data):
        feature_map = self.feature_map(data)
        avg_pool_output = torch.nn.functional.adaptive_avg_pool2d(input=feature_map, output_size = (1, 1))
        avg_output_flattened = torch.flatten(avg_pool_output)
        pred = self.model.fc(avg_output_flattened)
        return pred, feature_map

In [ ]:
model = PneumoniaModel.load_from_checkpoint("/kaggle/working/saved_models/epoch=0-step=375.ckpt", strict = False)
model.eval();

In [ ]:
def cam(model, img):
    with torch.no_grad():
        pred, features = model(img.unsqueeze(0))
    features = features.reshape((512, 49))
    weight_params = list(model.model.fc.parameters())[0]
    weight = weight_params[0].detach()
    
    cam = torch.matmul(weight, features)
    cam_img = cam.reshape(7, 7).cpu()
    return cam_img, torch.sigmoid(pred)

In [ ]:
def visualize(img, cam, pred):
    img = img[0]
    cam = transforms.functional.resize(cam.unsqueeze(0), (224, 224))[0]
    
    fig, axis = plt.subplots(1, 2)
    axis[0].imshow(img, cmap = 'bone')
    axis[1].imshow(img, cmap= 'bone')
    axis[1].imshow(cam, alpha=0.5, cmap="jet")
    plt.title(pred>0.5)

In [ ]:
val_dataset[-6][0].shape

In [ ]:
img = torch.cat([val_dataset[-6][0], val_dataset[-6][0], val_dataset[-6][0]], dim=0)
activation_map, pred = cam(model, img)

In [ ]:
visualize(img, activation_map, pred)

In [ ]:
img = torch.cat([val_dataset[-50][0], val_dataset[-50][0], val_dataset[-50][0]], dim=0)
activation_map, pred = cam(model, img)
visualize(img, activation_map, pred)

In [ ]:
img = torch.cat([val_dataset[50][0], val_dataset[50][0], val_dataset[50][0]], dim=0)
activation_map, pred = cam(model, img)
visualize(img, activation_map, pred)

In [ ]:
img = torch.cat([val_dataset[200][0], val_dataset[200][0], val_dataset[200][0]], dim=0)
activation_map, pred = cam(model, img)
visualize(img, activation_map, pred)